# Modeling a modulus in atomistic detail



In this exercise you will develop a simple, _atomic scale_ model of how metals respond under tensile stress. Simulating the response in such detail will allow you an insight into how dislocations are actually generated and interact, which is the basis for the plastic response of the material. Large scale plasticity is of course more complicated, and depends on the interactions of an immense number of dislocations as well as grain boundaries and more rigid inclusions. 

In the course of the exercise, you will:

1. Create and manipulate atomic scale structural models of crystalline matter
1. Compute simple elastic properties of single crystal copper (Cu)




### Milestone 1

Basic use of ASE, computation of the elastic properties of a single crystal of copper. Do the tasks right here in this notebook. Bring the completed notebook to the a marking session, at latest by the middle of Term.

### Task 1.1

You are going to use a simple model to describe the interaction of atoms, called the "Morse potential", 
which assumes that atoms interact with each other pairwise, and the total potential energy of a collection of atoms is

$$
E_{total} = \sum_{i < j} V_{M}(r_{ij})
$$

where the double sum runs over the indices of every pair of atoms (counting every pair only once), $r_{ij}$ is the distance between atoms $i$ and $j$, and the _interaction potential_ is given by

$$
V_{M}(r) = D \left[e^{-2\alpha (r-r_0))} - 2e^{-\alpha (r-r_0))} \right]
$$

There are three adjustable parameters in this model. You can think of $D$ as fixing the energy scale of the model, $r_0$ as fixing the distance between neighbouring atoms, and $\alpha$ as fixing the length scale over which the interaction between atoms decays (in inverse length units). 

In the small scale world of atoms, it is convenient to measure energies in _electron Volts_ (eV), with 1 eV $\approx 1.6 \times 10^{-19}~$J, and distances in _Ångstroms_ (Å), with 1 Å = $10^{-10}~$m, and these are the intrinsic units of ASE. All other derived units follow from this, e.g. forces are in eV/Å, stresses are in eV/Å$^3$. ASE provides the constant ```GPa``` that can be used to convert pressures and stresses from eV/Å$^3$ into GPa units. 

Here is how to create an object that represents just two copper atoms with a distance 2.5 Å between them:

In [ ]:
!pip install ase

In [ ]:
!cp "drive/My Drive/Morse.py" .

In [ ]:
from __future__ import print_function
import numpy as np
from ase import Atoms
from ase.units import eV, Ang, GPa
d = 2.5*Ang
a = Atoms('2Cu', positions=[(0., 0., 0.), (0., 0., d)])

The ```positions``` array contains the x, y, z coordinates of each atom, in that order. 

Now attach a ```calculator``` (ASE-speak for a model that can compute energies, forces and stresses on atoms) to this object, and compute its potential energy. (Any time you create a new atoms object, you need to set its calculator, but you can reuse the same calculator object)

In [ ]:
import Morse
calc = Morse.MorsePotential()

a.set_calculator(calc) 
a.get_potential_energy()

Changing the position of the second atom allows the evaluation of the potential energy as a function of distance

In [ ]:
p = a.get_positions()
p

In [ ]:
p[1,2] = 2.8
p

In [ ]:
a.set_positions(p)
a.get_potential_energy()

You can also manipulate the positions of the atoms directly by access the array inside the ```Atoms``` object:

In [ ]:
a.positions[1,2] = 2.9
a.get_positions()

The forces exerted by the atoms on one another can be obtained analogously. Remember that the force is the negative of the 3-dimensional gradient vector of the potential energy. (But there is no directly accessible array in the ```Atoms``` object that holds the forces, you have to use the function call)

In [ ]:
f = a.get_forces()
print(np.linalg.norm(f[0]))


### Deliverable 1.1

Write a function that computes the Morse potential energy for two atoms for a given distance between them, and use it to create a plot of the energy against distance. Now do the same for the magnitude of the force exerted by one atom on the other.

What is the distance between the two atoms corresponding to the lowest potential energy ?


In [ ]:
import math
import matplotlib.pyplot as plt
%matplotlib inline  
calc = Morse.MorsePotential()
    
def potential_energy(d):
    
    a = Atoms('2Cu', positions=[(0., 0., 0.), (0., 0., d*Ang)])
    calc = Morse.MorsePotential()
    a.set_calculator(calc) 
    V =a.get_potential_energy()
        
    return V 

V=[]
distances=np.arange(2,5,0.1)
for i in distances:
    V.append(potential_energy(i))

print(np.min(V))
print(np.argmin(V))
print(distances[6])

plt.plot(distances,V)
plt.xlabel("distance/A")
plt.ylabel("pe/EV")
plt.show

using the function above for graphs

In [ ]:
def Force(d):
    
    a = Atoms('2Cu', positions=[(0., 0., 0.), (0., 0., d*Ang)])
    a.set_calculator(calc) 
    f=a.get_forces()
        
    return f

F=[]
distances=np.arange(2,5,0.05)
for i in distances:
    F.append(1/(math.sqrt(2))*np.linalg.norm(Force(i)))


plt.plot(distances,F)


### Deliverable 1.2 

Write a _unit test_ that verifies that the forces returned by the ```get_forces()``` function is really the negative gradient of the energy (as returned by the ```get_potential_energy()``` function) with respect to the atomic positions. Do this by comparing the returned forces to those obtained by numerical finite differences of energies between two sets of atomic positions, displaced by small amount.

Hint:

Use the idea of the definition of the gradient and its relation to the Taylor expansion:

$$
\nabla f(x) \approx \frac{f(x+\epsilon)-f(x)}{\epsilon}
$$

With just two atoms in the "system", you can treat the energy and force as just functions of a scalar variable, the distance between the two atoms. More generally, both the energy and the forces are functions of many atomic coordinates, which we can collect into a vector $\bf{R}$, so the truncated Taylor expansion takes the form:

$$
\nabla f({\bf R})\cdot {\bf d} \approx \frac{f ({\bf R}+\epsilon {\bf d} )-f({\bf R})}{\epsilon}
$$

where ${\bf d}$ is the vector representing a small displacement. 

__Experiment with different values of $\epsilon$ and look at how accurate the approximation is as a function of $\epsilon$. What goes wrong if you make $\epsilon$ too small?__

In [ ]:
e=0.0000000001
d=4

def grad_energy(d,e):
    gradient = (potential_energy(d+e)-potential_energy(d))/e
    return gradient

x=(Force(d)[1][2])
y=grad_energy(d,e)
print (x)
print(y)
assert(-x-y <0.000001)

### Deliverable 1.3

_Create a cubic unit cell of the Cu crystal, and extract some of its properties._

In order to investigate the properties of bulk copper, we need to model a large number of atoms. But evaluating the total potential energy of a large number of atoms takes a long time, and so we need a shortcut. In fact many simple properties of a crystalline solid can be evaluated by just considering its _unit cell_, i.e. the smallest repeating unit from which the crystal is made. This is typically true for static properties that do not depend on atoms experiencing a variety of neighbour environments, i.e. in the absence of _defects_. Such properties include the _lattice constant_ (i.e. the density), the _equation of state_, the _elastic constants_ (including the _bulk modulus_), the _Poisson ratio_, etc. 

The trick is to _assume_ that the atoms are arranged in perfect crystalline order, and only explicitly consider those atoms that are in a single unit cell. In order to correctly evaluate the energy, the _effect_ of atoms in neighbouring unit cells needs to be included, but this can be done by using [_periodic boundary conditions_](https://en.wikipedia.org/wiki/Periodic_boundary_conditions). 

In the image below, the blue shaded area is the unit cell, its sides are the lattice vectors that generate the periodic images. 

More complicated properties, such as the _yield stress_, or those of more complicated materials such as _polycrystalline solids_ cannot be extracted from such small unit cells.

The crystal structure of Cu is face centered cubic, and ASE provides convenient constructors which assume periodic boundary conditions, unless instructed otherwise:

In [ ]:
from ase.build import bulk
cu = bulk("Cu", "fcc", a=3.6, cubic=True)

Although the smallest possible unit cell of an fcc crystal contains just one atom, it is often more convenient to work with the "cubic unit cell" which contains 4 atoms, but whose lattice vectors (which represent the displacements that correspond to the periodically repeating units) are just the sides of a cube, i.e. orthogonal and correspond to the x, y, and z axes. The size of the lattice vectors determine the volume of the unit cell and the corresponding density of material modelled. 

The fcc crystal you have seen before probably show an atom in each face of the cubic unit cell - but many of those are repeated periodic images of the ones shown above!

In ASE, the lattice vectors are stored, side-by-side, collected together in a 3x3 matrix called _cell_. So, the first column is the first lattice vector ($L_1$), the second column is the second lattice vector($L_2$), and the third column is the third lattice vector ($L_3$).

$$
\left[ \begin{matrix} L_{1x} &L_{2x} &L_{3x}\\ L_{1y} &L_{2y}&L_{3y}\\ L_{1z} &L_{2z}&L_{3z} \end{matrix}\right]
$$

In [ ]:
cu.get_cell()

There are indeed 4 atoms in this unit cell, with positions corresponding to the origin, and the centers of the sides of the unit cube on the YZ, ZX and XY planes:

In [ ]:
cu.get_cell()

Verify that the periodic boundary conditions are implemented correctly by evaluating the potential energy of the unit cell, and comparing the potential energy per atom to that of a larger cell, which is obtained by replicating the original twice in each of the three directions. Since both smaller and larger cells represent an infinite, periodic array of copper atoms, the potential energy per atom should be the same. 

In [ ]:
cu.set_calculator(calc)
print("Number of atoms: ", cu.get_number_of_atoms())
print("Potential energy per atom: ", cu.get_potential_energy()/cu.get_number_of_atoms())

In [ ]:
cu222 = cu.copy()        # creating a copy of an Atoms object
cu222.set_calculator(calc) # copying DOES NOT bring the attached calculator, so we need to set it again
cu222 *= (2,2,2)         # replicating the unit cell is accomplished by the multiplying operator
print("Number of atoms: ", cu222.get_number_of_atoms())
print("Potential energy per atom: ", cu222.get_potential_energy()/cu222.get_number_of_atoms())

Strain can be applied to the system by modifying (distorting) the unit cell appropriately. E.g. to apply 1% hydrostatic compression:

In [ ]:
cell = cu.get_cell()
cell *= 0.99
cu.set_cell(cell, scale_atoms=True) # To apply strain, the atomic positions need to be scaled together with the unit cell 
cu.get_cell()


In [ ]:
cu.get_potential_energy()/cu.get_number_of_atoms()

The stress on the system is given as a matrix, here we get it as a 3x3 matrix (ASE's default is to provide it in [Voigt notation](https://en.wikipedia.org/wiki/Voigt_notation))

In [ ]:
cu.get_stress(voigt=False)


The stress, analogously to the force, is the derivative of the energy with respect to the unit cell vectors (or more precisely, with respect to the deformation strain applied to the unit cell vectors). Note how the off-diagonal elements of the stress matrix above are essentially zero and the diagonal elements are the same, i.e. the stress is the same in the X, Y, and Z directions, corresponding to the 1% hydrostatic compression that we applied above. 

### Deliverable 1.4

- __Write a program that calculates and plots the potential energy and pressure (P) of the copper crystal as a function of volume by applying varying amounts of hydrostatic strain.__
- __Calculate the [bulk modulus](https://en.wikipedia.org/wiki/Bulk_modulus) at the equilibrium volume. Compare it to the experimental value.__

Hints:

- Make sure you use a sensible range of strains, think about what would be reasonable in a real experiment
- For the plot, use both compressive and tensile strains. Make sure you plot against the volume, not against the strain itself.
- Remember that $K = -V dP/dV$ where $V$ is the volume, and that the pressure is related to the trace of the stress matrix (S), $P = - \frac{1}{3} \text{Tr}(S)$.
- Do not forget that energy and volume are extensive, so it is best to work with the per-atom quantities.
- Do not be surprised that you only get within the experimental value within 20% or so. 

In [ ]:
cu = bulk("Cu", "fcc", a=3.6, cubic=True)
cu.set_calculator(calc)

def potential_energy():    
    return cu.get_potential_energy()/cu.get_number_of_atoms()


strain = np.arange(0.9,1.05,0.05)
volume=[]
energy=[]
for i in strain:
    cu = bulk("Cu", "fcc", a=3.6, cubic=True)
    cu.set_calculator(calc)
    cell=cu.get_cell()
    cell*=i
    cu.set_cell(cell,scale_atoms=True)
    volume.append(cu.get_volume()/cu.get_number_of_atoms())
    energy.append(potential_energy())

from ase.eos import EquationOfState
from ase.units import kJ
eos = EquationOfState(volume, energy, eos="birchmurnaghan") # Birch-Murnaghan is a particular functional form fitted to the equation of state
v0, e0, B = eos.fit()
print('Bulk modulus: ', B / kJ * 1.0e24, 'GPa')
eos.plot()  

In [ ]:
cu = bulk("Cu", "fcc", a=3.6, cubic=True)
cu.set_calculator(calc)
print(cu.get_stress(voigt=False))
def pressure():
    p = cu.get_stress(voigt=False)
    return p[0][0]

strain = np.arange(0.9,1.5,0.1)
volume=[]
p=[]
for i in strain:
    cu = bulk("Cu", "fcc", a=3.6, cubic=True)
    cu.set_calculator(calc)
    cell=cu.get_cell()
    cell*=i
    cu.set_cell(cell, scale_atoms=True)
    volume.append(cu.get_volume())
    p.append(pressure())
    
plt.plot(volume,p,'b-')
plt.xlabel('volume of unit cell')
plt.ylabel('stresses')
plt.show
    

### Deliverable 1.5 

__Write a program to compute the [shear modulus](https://en.wikipedia.org/wiki/Shear_modulus).__

Hint: You can apply a shear in the XY plane by modifying the X component of the lattice vector that points originally in the Y direction, i.e. $L_{2x}$. Apply _small_ amounts of shear, i.e. a few percent. Starting with a cubic cell with side lengths L, new cell matrix, after applying 1% shear in the XY plane, would be given by

$$
\left[\begin{matrix}
L & 0.01L & 0\\0&L&0\\ 0&0&L
\end{matrix}\right]
$$

Just as before, in the hydrostatic case, after applying the shear strain in this way, you can get the corresponding (shear) stress simply by calling the ```get_stress()``` function. Observe which components of the stress matrix is nonzero, and think about which one you need to work out the shear modulus. 

The computed shear modulus of this model deviates from the experimental value by quite a bit more than the bulk modulus - this is typical for such simple models. 

__Write a program to determine the [Poisson ratio](https://en.wikipedia.org/wiki/Poisson%27s_ratio)__

Hint: Recall that if the system is strained in the X direction by a normal strain (not shear), the resulting stress will not be purely in the X direction, but in addition to the stress in X there will be stress in the Y and Z directions also. If you now strain the system in an _equibiaxial_ way, so some amount of strain in X, and equal amounts in Y and Z, that is different from the strain in X, the Poisson ratio is the ratio of X and Y(and Z) strains that lead to no stress in the Y (and Z) directions. 

- Think about the sign of the X strain and the Y(and Z) strains.
- Make sure you use a sensible range of strains. 
- The Poisson ratio depends very sensitively on the starting volume, so make sure you use the unit cell size that corresponds to the lowest energy, accurate to at least 4 decimal places. 
- You should get a computed value that is within 10% of the experimentally observed one. 


#### Comparison to experiment

You will note that there are significant discrepancies from the experimental values of the elastic properties. Some of this is due to the simplicity of the Morse potential, but not all of it. 

- The simple calculations you did apply to a "single crystal" of the material, where we assume that the entire extent of the material is just periodic copies of the same unit cell. In reality a typical piece of copper would have many grains, each of which are single crystals, but they are oriented randomly, so anything you measure on the sample is a rotationally averaged quantity. 
- It turns out that the elastic properties do depend on the orientation, so our single crystal model behaves quite differently from a rotationally averaged system.
- For example, the simple relationship that you know between the bulk modulus, shear modulus and the Poisson ratio only holds for isotropic materials (so including polycrystalline materials, because of the rotational averaging), and we don't expect it to hold for the single crystal.
- What is the deviation of the above results from the simple relationship between the three elastic properties? Which is the one that is most different from the experimental polycrystalline value? 

In [ ]:
shear_strain=np.arange(0,0.1,0.01)
t=[]#taw
volume=[]
for i in shear_strain:
    cu = bulk("Cu", "fcc", a=3.6, cubic=True)
    cu.set_calculator(calc)
    cell = cu.get_cell()
    cell[0][1]=3.6*i
    cu.set_cell(cell, scale_atoms=True)
    x=(cu.get_stress(voigt=False))
    t.append(x[0][1])

print (x)
print ("shear modulus is : " )

print (t[1]/0.01/GPa)
plt.plot(shear_strain,t,'b-')
plt.xlabel('strain')
plt.ylabel('stresses')
plt.show

In [ ]:
from ase.build import bulk
eq = 3.60288
cu = bulk("Cu", "fcc", a=eq, cubic=True)
cu.set_calculator(calc)
cell = cu.get_cell()
n = (cu.get_number_of_atoms())

x_strain = 0.01
#print (cu.get_stress(voigt=False))

cell[0][0] = cell[0][0]*(1+x_strain)

for i in np.arange(x_strain*0.01,x_strain*0.4,x_strain/100):
    cell[1][1] = (eq)*(1-i)
    cell[2][2] = (eq)*(1-i)
    cu.set_cell(cell)
   
    #print (cu.get_stress(voigt = False))
    #print (i,cu.get_stress(voigt=False)[1,1])
    
    if cu.get_stress(voigt=False)[1][1] < 0:
        print ("v = ",i/x_strain)
        break
